<a href="https://colab.research.google.com/github/aalexakos/thesis-smart-grids/blob/alex/campare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import datetime
import time
import numpy as np
from ast import literal_eval

pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/datasets

1620864000000_661403b2-9177-4451-a7d9-9ab3df7ea756.csv
1621036800000_e20fcd7d-9c61-4cc0-ad0a-1bda8ddbdd0d.csv
1621123200000_39c690cb-82a1-4be0-989b-3fd9a44601c0.csv


In [2]:
#---------------------data from ai prediction-----------------------
csv=('/content/drive/MyDrive/datasets/1622246400000_a404a4bd-93e5-407b-9a43-514e3608160b.csv') 

df=pd.read_csv(csv)
true_irradiance=df.loc[df['sensor_id'] == 'irradiance']

#add human time in the dataframe
true_irradiance['datetime']=(pd.to_datetime(true_irradiance['timestamp'],unit='ms'))

#sort data frame based in epoch time
true_irradiance=true_irradiance.sort_values('timestamp')

#round datetime to the closest hour in order to compare to my data
true_irradiance['datetime'] = true_irradiance['datetime'].dt.ceil('h')

#round timestamp to closest hour based in datetime
true_irradiance['timestamp'] = true_irradiance['datetime'].astype('int64')

#removing milliseconds from timestamp
true_irradiance['timestamp']=true_irradiance['timestamp'].floordiv(1000000000)

#seperated device id
true_irradiance = true_irradiance.pivot_table(index=['gateway_id',	'sensor_id',	'timestamp', 'deployment_id',	'datetime'], columns='device_id' ,values='value', aggfunc='first').reset_index()
true_irradiance.reset_index(drop=True, inplace=True)
true_irradiance


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


device_id,gateway_id,sensor_id,timestamp,deployment_id,datetime,meteo-J01,meteo-J02,meteo-J04
0,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622174400,HELIOS,2021-05-28 04:00:00,17.27,1.61,20.03
1,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622178000,HELIOS,2021-05-28 05:00:00,28.00,34.94,33.25
2,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622181600,HELIOS,2021-05-28 06:00:00,148.12,110.76,178.92
3,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622185200,HELIOS,2021-05-28 07:00:00,372.08,324.67,411.44
4,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622188800,HELIOS,2021-05-28 08:00:00,593.00,541.07,632.18
5,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622192400,HELIOS,2021-05-28 09:00:00,775.17,732.54,825.60
6,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622196000,HELIOS,2021-05-28 10:00:00,919.72,880.22,947.02
7,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622199600,HELIOS,2021-05-28 11:00:00,991.81,962.31,1009.11
8,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622203200,HELIOS,2021-05-28 12:00:00,979.23,968.35,988.60
9,HELIOS_mapper-p2z7tix9jogx0yxl,irradiance,1622206800,HELIOS,2021-05-28 13:00:00,905.91,909.59,904.01


In [3]:
#------------------radiation from openweathermap forecast prediction----------------------
csv=('/content/drive/MyDrive/datasets/dailyRadiationForecast28052021.csv') 

openweathermap=pd.read_csv(csv,index_col=0)


#add human time in the dataframe
openweathermap['datetime']=(pd.to_datetime(openweathermap['dt'],unit='s'))
openweathermap = openweathermap.rename({'dt': 'timestamp'}, axis=1)

openweathermap.radiation=openweathermap.radiation.apply(literal_eval)
value = [d.get('ghi') for d in openweathermap.radiation]
openweathermap['radiation']=pd.DataFrame({'radiation':value})
openweathermap

,radiation,timestamp,datetime
0,169.51,1622178000,2021-05-28 05:00:00
1,364.82,1622181600,2021-05-28 06:00:00
2,561.58,1622185200,2021-05-28 07:00:00
3,737.97,1622188800,2021-05-28 08:00:00
4,877.38,1622192400,2021-05-28 09:00:00
5,967.49,1622196000,2021-05-28 10:00:00
6,1000.57,1622199600,2021-05-28 11:00:00
7,973.80,1622203200,2021-05-28 12:00:00
8,889.47,1622206800,2021-05-28 13:00:00
9,754.82,1622210400,2021-05-28 14:00:00


In [4]:
#------------------radiation from clima cell forecast prediction----------------------
csv=('/content/drive/MyDrive/datasets/dailyClimaCellForecast28052021.csv') 

climacell=pd.read_csv(csv,index_col=0)
climacell['startTime']

#add epoch time in the dataframe
climacell['timestamp']=climacell['startTime'].astype('datetime64[s]')
climacell['timestamp']=climacell['timestamp'].astype('int64')
#removing milliseconds from timestamp
climacell['timestamp']=climacell['timestamp'].floordiv(1000000000)


climacell.values=climacell['values'].apply(literal_eval)
value = [d.get('solarGHI') for d in climacell['values']]
climacell['values']=pd.DataFrame({'radiation':value})

climacell

,startTime,values,timestamp
0,2021-05-28T05:00:00Z,143.41,1622178000
1,2021-05-28T06:00:00Z,326.75,1622181600
2,2021-05-28T07:00:00Z,514.78,1622185200
3,2021-05-28T08:00:00Z,673.67,1622188800
4,2021-05-28T09:00:00Z,811.22,1622192400
5,2021-05-28T10:00:00Z,904.86,1622196000
6,2021-05-28T11:00:00Z,947.49,1622199600
7,2021-05-28T12:00:00Z,931.39,1622203200
8,2021-05-28T13:00:00Z,827.98,1622206800
9,2021-05-28T14:00:00Z,701.82,1622210400


In [12]:
comb=pd.merge(true_irradiance, openweathermap, on="timestamp")
combined=pd.merge(comb, climacell, on="timestamp", how = 'left')
combined.drop(['sensor_id', 'deployment_id', 'datetime_y', 'startTime'],axis=1, inplace=True)
combined.dropna()
combined.rename(columns={'values': 'climacell', 'radiation':'openweathermap'}, inplace=True)
combined.to_csv('data.csv')
!cp data.csv "drive/My Drive/"
combined

,gateway_id,timestamp,datetime_x,meteo-J01,meteo-J02,meteo-J04,openweathermap,climacell
0,HELIOS_mapper-p2z7tix9jogx0yxl,1622178000,2021-05-28 05:00:00,28.00,34.94,33.25,169.51,143.41
1,HELIOS_mapper-p2z7tix9jogx0yxl,1622181600,2021-05-28 06:00:00,148.12,110.76,178.92,364.82,326.75
2,HELIOS_mapper-p2z7tix9jogx0yxl,1622185200,2021-05-28 07:00:00,372.08,324.67,411.44,561.58,514.78
3,HELIOS_mapper-p2z7tix9jogx0yxl,1622188800,2021-05-28 08:00:00,593.00,541.07,632.18,737.97,673.67
4,HELIOS_mapper-p2z7tix9jogx0yxl,1622192400,2021-05-28 09:00:00,775.17,732.54,825.60,877.38,811.22
5,HELIOS_mapper-p2z7tix9jogx0yxl,1622196000,2021-05-28 10:00:00,919.72,880.22,947.02,967.49,904.86
6,HELIOS_mapper-p2z7tix9jogx0yxl,1622199600,2021-05-28 11:00:00,991.81,962.31,1009.11,1000.57,947.49
7,HELIOS_mapper-p2z7tix9jogx0yxl,1622203200,2021-05-28 12:00:00,979.23,968.35,988.60,973.80,931.39
8,HELIOS_mapper-p2z7tix9jogx0yxl,1622206800,2021-05-28 13:00:00,905.91,909.59,904.01,889.47,827.98
9,HELIOS_mapper-p2z7tix9jogx0yxl,1622210400,2021-05-28 14:00:00,769.90,781.78,754.11,754.82,701.82


In [9]:
combined.corr(method='pearson')

,timestamp,meteo-J01,meteo-J02,meteo-J04,openweathermap,climacell
timestamp,1.000000,-0.433843,-0.400022,-0.463756,-0.585071,-0.574411
meteo-J01,-0.433843,1.000000,0.997692,0.998352,0.981457,0.984085
meteo-J02,-0.400022,0.997692,1.000000,0.992260,0.970727,0.972999
meteo-J04,-0.463756,0.998352,0.992260,1.000000,0.987377,0.990218
openweathermap,-0.585071,0.981457,0.970727,0.987377,1.000000,0.999457
climacell,-0.574411,0.984085,0.972999,0.990218,0.999457,1.000000


In [11]:
combined.corr(method='kendall')

,timestamp,meteo-J01,meteo-J02,meteo-J04,openweathermap,climacell
timestamp,1.000000,-0.422463,-0.367215,-0.450980,-0.420084,-0.420084
meteo-J01,-0.422463,1.000000,0.900682,0.976946,0.976481,0.976481
meteo-J02,-0.367215,0.900682,1.000000,0.918038,0.903136,0.903136
meteo-J04,-0.450980,0.976946,0.918038,1.000000,0.980196,0.980196
openweathermap,-0.420084,0.976481,0.903136,0.980196,1.000000,1.000000
climacell,-0.420084,0.976481,0.903136,0.980196,1.000000,1.000000


In [13]:
combined.corr(method='spearman')

,timestamp,meteo-J01,meteo-J02,meteo-J04,openweathermap,climacell
timestamp,1.000000,-0.551190,-0.505937,-0.581011,-0.573659,-0.573659
meteo-J01,-0.551190,1.000000,0.983964,0.995866,0.994807,0.994807
meteo-J02,-0.505937,0.983964,1.000000,0.987094,0.982886,0.982886
meteo-J04,-0.581011,0.995866,0.987094,1.000000,0.994827,0.994827
openweathermap,-0.573659,0.994807,0.982886,0.994827,1.000000,1.000000
climacell,-0.573659,0.994807,0.982886,0.994827,1.000000,1.000000
